In [1]:
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

### Train

In [2]:
train = pd.read_csv('../../ml-data/cyberattack_detection/3_selected_features/A1_train_data_PCA.csv', sep=',')

In [3]:
train.head()

,PC1,PC2,PC3
0,277104.332320,17405.523770,-13546.742781
1,-150864.801811,-435.860958,-43929.108886
2,-120787.105699,572.759831,-43403.206845
3,254876.957762,14623.397761,-36966.671201
4,-117895.439420,621.683934,-43348.286842


In [4]:
# set contamination = 0.02
isolation_forest = IsolationForest(contamination=0.02)

In [5]:
isolation_forest.fit(train)

IsolationForest(contamination=0.02)

### Test

In [6]:
test = pd.read_csv('../../ml-data/cyberattack_detection/3_selected_features/A1_test_data_PCA.csv', sep=',')

In [7]:
test.shape

(1053845, 3)

In [8]:
test_predict = isolation_forest.predict(test)
decision_function = isolation_forest.decision_function(test)

In [9]:
test['anomaly'] = test_predict
test['score'] = decision_function

In [10]:
test['anomaly'].value_counts()

 1    1031987
-1      21858
Name: anomaly, dtype: int64

In [11]:
test.loc[test['anomaly'] == -1]

,PC1,PC2,PC3,anomaly,score
8392,30242.508368,-116872.281274,-36952.183835,-1,-0.022110
8400,162345.455384,-111952.366025,-34686.857240,-1,-0.005308
8402,107850.267029,-113981.932467,-35621.351150,-1,-0.012944
8405,-30379.529421,-119130.030211,-37991.742131,-1,-0.024484
8407,64078.064344,-115612.142333,-36371.965252,-1,-0.015361
...,...,...,...,...,...
924067,-74575.445432,-103562.370388,29723.928605,-1,-0.025922
924068,-74848.630193,-95910.486418,29023.945043,-1,-0.021154
924069,-74570.721878,-103694.676754,29736.031805,-1,-0.025922
1004000,386700.065874,-105382.110815,-49725.463993,-1,-0.025810


In [ ]:
sns.pairplot(test, hue = 'anomaly')

In [13]:
anomalies = test.loc[test['anomaly'] == -1]

In [14]:
anomalies.head()

,PC1,PC2,PC3,anomaly,score
8392,30242.508368,-116872.281274,-36952.183835,-1,-0.022110
8400,162345.455384,-111952.366025,-34686.857240,-1,-0.005308
8402,107850.267029,-113981.932467,-35621.351150,-1,-0.012944
8405,-30379.529421,-119130.030211,-37991.742131,-1,-0.024484
8407,64078.064344,-115612.142333,-36371.965252,-1,-0.015361


In [15]:
original_test = pd.read_csv('../../ml-data/cyberattack_detection/1_raw_data/A2_1/clean_test_data', sep=',')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
anomalies_orig_feat = original_test[original_test.index.isin(anomalies.index)]

In [17]:
anomalies_orig_feat = anomalies_orig_feat.merge(anomalies[['score']], left_index = True, right_index=True)

In [ ]:
anomalies_orig_feat.head(40)

In [18]:
anomalies_orig_feat['SrcIPAddress'].nunique()

24

In [19]:
import numpy as np
anomalies_orig_feat.loc[anomalies_orig_feat['PacketsSec'] != np.inf].describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))


,Duration,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,score
count,3258,3258,159,3258,3258,3258,3258,3258,3258,3258
mean,940.563,0,0,73.5003,74716.9,75371.9,4413.67,207520,103321,-0.0472533
std,693.538,0,0,611.292,647197,653236,31397.2,1.40667e+06,706699,0.0238678
min,4e-06,0,0,2,114,57,0.000707076,0.0766297,0.0743164,-0.0709403
25%,835.848,0,0,3,2108,2104,0.00336618,2.28381,2.27792,-0.0679823
50%,959.792,0,0,12,8281,8263.5,0.0165886,13.4119,13.4166,-0.0679823
75%,1098.46,0,0,26,22107.2,22389.2,0.040342,40.7493,40.6305,-0.0237403
max,3759.24,0,0,8947,9.45955e+06,9.49176e+06,750000,3.05e+07,1.55e+07,-0.000279514


In [20]:
anomalies_orig_feat.to_csv('../4 Results/A1_test_iForest_PCA_0.02_orig_feat.csv', sep=',')

End